In [1]:
import re
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D

Using TensorFlow backend.


# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev1 toc-item"><a href="#Getting-the-data" data-toc-modified-id="Getting-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the data</a></div><div class="lev1 toc-item"><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></div><div class="lev2 toc-item"><a href="#Splitting-a-sentence-of-arbitrary-length-into-fixed-input-size" data-toc-modified-id="Splitting-a-sentence-of-arbitrary-length-into-fixed-input-size-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Splitting a sentence of arbitrary length into fixed input size</a></div><div class="lev2 toc-item"><a href="#Turning-words-into-integers" data-toc-modified-id="Turning-words-into-integers-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Turning words into integers</a></div><div class="lev1 toc-item"><a href="#Constructing-the-targets-from-sentences" data-toc-modified-id="Constructing-the-targets-from-sentences-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constructing the targets from sentences</a></div><div class="lev1 toc-item"><a href="#Constructing-the-train/dev/test-sets" data-toc-modified-id="Constructing-the-train/dev/test-sets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Constructing the train/dev/test sets</a></div><div class="lev1 toc-item"><a href="#Ideas-for-the-architecture" data-toc-modified-id="Ideas-for-the-architecture-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ideas for the architecture</a></div><div class="lev2 toc-item"><a href="#The-initial-architecture" data-toc-modified-id="The-initial-architecture-61"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>The initial architecture</a></div><div class="lev2 toc-item"><a href="#Training-the-model" data-toc-modified-id="Training-the-model-62"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Training the model</a></div><div class="lev1 toc-item"><a href="#pre-processing-pipeline" data-toc-modified-id="pre-processing-pipeline-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>pre-processing pipeline</a></div><div class="lev1 toc-item"><a href="#Evaluation-metrics" data-toc-modified-id="Evaluation-metrics-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluation metrics</a></div><div class="lev1 toc-item"><a href="#Deployment" data-toc-modified-id="Deployment-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Deployment</a></div>

# Introduction
I want to build a neural network that takes a danish sentence as an input and suggests if and where commas should be placed in the sentence.

# Getting the data
It should be possible to get a corpus of danish sentence somewhere on the internet. I have to be careful about the source, since several different comma rules can be used, and I don't want to confuse the network with a mixture of the different rules.

My raw data should be a set of sentence like the following:

In [2]:
example_1 = "NEW YORK – Mens mediernes bevågenhed i denne uge var rettet mod Houston, \
        hvor stormen Harvey og de efterfølgende oversvømmelser har kostet mindst \
        39 livet, var der fire nye og potentielt væsentlige udviklinger i den løbende \
        efterforskning af forbindelserne mellem Rusland og Donald Trumps præsidentkampagnestab."

In [3]:
example_2 = "Af folk der ikke kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe spærrer vejen."

# Preprocessing
The pre-cprocessing will include a couple of steps.
1. Taking the full sentences and cutting them into pre-defined lengths
1. Turning the words into integer representations (using the top-n words)

## Splitting a sentence of arbitrary length into fixed input size
There will be quite a few decisions from the pre-processing that will probably have to be optimized like hyper-parameters later. For now I have just guessed on some values that I think will be decent.

In [4]:
def get_word_scopes(sentence):
    word_scopes = []
    word_scope = 10
    step_size = 2
    off_set = 5
    words = sentence.split()
    num_words = len(words)

    st= 0
    nd = st + word_scope
    while nd<num_words+step_size:
        temp_sentence = ' '.join(words[st:nd])
        st = st + step_size
        nd = nd + step_size
        word_scopes.append(temp_sentence)
    return word_scopes

In [5]:
get_word_scopes(example_1)

['NEW YORK – Mens mediernes bevågenhed i denne uge var',
 '– Mens mediernes bevågenhed i denne uge var rettet mod',
 'mediernes bevågenhed i denne uge var rettet mod Houston, hvor',
 'i denne uge var rettet mod Houston, hvor stormen Harvey',
 'uge var rettet mod Houston, hvor stormen Harvey og de',
 'rettet mod Houston, hvor stormen Harvey og de efterfølgende oversvømmelser',
 'Houston, hvor stormen Harvey og de efterfølgende oversvømmelser har kostet',
 'stormen Harvey og de efterfølgende oversvømmelser har kostet mindst 39',
 'og de efterfølgende oversvømmelser har kostet mindst 39 livet, var',
 'efterfølgende oversvømmelser har kostet mindst 39 livet, var der fire',
 'har kostet mindst 39 livet, var der fire nye og',
 'mindst 39 livet, var der fire nye og potentielt væsentlige',
 'livet, var der fire nye og potentielt væsentlige udviklinger i',
 'der fire nye og potentielt væsentlige udviklinger i den løbende',
 'nye og potentielt væsentlige udviklinger i den løbende efterforskning 

In [6]:
get_word_scopes(example_2)

['Af folk der ikke kan komme væk fra deres egen',
 'der ikke kan komme væk fra deres egen tragedie, fordi',
 'kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe',
 'væk fra deres egen tragedie, fordi amerikanske krigsskibe spærrer vejen.']

## Turning words into integers
For this I need a list of the most frequently occurring words in the danish language. If my corpus is big enough, I can construct it from there, but it should be possible to find a list somewhere on the internet. Below I construct the dictionary from the corpus.

In [7]:
with open('corpera/europarl_raw/danish/ep-00-01-17.txt') as f:
    sentences = f.readlines()

In [8]:
len(sentences)

962

In [9]:
#sentences = [example_1, example_2 ]
corpus = ' '.join(sentences)
c = Counter(corpus.split())
n_words = 5000
most_common_words = c.most_common(n_words)
top_n_words = {}
for idx, item in enumerate(most_common_words):
    word = item[0]
    top_n_words[word] = idx

Once I have the corpus I can do the word embedding.

In [10]:
def embed_word_scope(word_scope, top_n_words):
    """
        Input
        word scope: A string containing 10 words.
        top_n_words: A dictionary mapping words to integers.
        
        Returns
        embedded: A list of the word embeddings (interger representation) for word_scope.
    """
    embedded = []
    for word in word_scope.split():
        if word in top_n_words:
            word_idx = top_n_words[word]
            embedded.append(word_idx)
        else:
            embedded.append(0)
    return embedded

# Constructing the targets from sentences
I'm not completely sure about the architecture I am going to use, but I think that the I will have a fixed input score (say 10 words rather than the entire sentence), and then loop over the entire sentence. I will have an output layer with a neuron for each word in the scope, which will signify whether or not the word should be followed by a comma.

By splitting the sentence into individual words, we can find the index of the words that is followed by a comma.

In [11]:
def get_y(sentence):
    words = sentence.split()
    y = np.zeros((1, 10))
    for idx, word in enumerate(words):
        if ',' in word:
            y[0, idx] = 1
    return y


We have now constructed the target for the output neurons

In [12]:
word_scopes = get_word_scopes(example_2)
print(word_scopes[2])
print(get_y(word_scopes[2]))

kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe
[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


# Constructing the train/dev/test sets
With the functions defined above it should be possible to construct the data with something like the following function:

In [13]:
def pre_process(sentences):
    X = []
    Y = []
    for sentence in sentences:
        word_scopes = get_word_scopes(sentence)
        for word_scope in word_scopes:
            y = get_y(word_scope)
            embedded_word_scope = embed_word_scope(word_scope, top_n_words)
            embedded_word_scope = embedded_word_scope.replace(',', '')
            X.append(embedded_word_scope)
            Y.append(y)
    return X, Y

In [14]:
#sentences = [example_1, example_2 ]
X, Y = pre_process(sentences)

In [15]:
for idx in range(5):
    print('Input: ', X[idx])
    print('Target:', Y[idx])

Input:  [1688, 7, 1689, 28, 1690, 410, 1691, 0, 17, 76]
Target: [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]
Input:  [1689, 28, 1690, 410, 1691, 0, 17, 76, 1692, 1693]
Target: [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]]
Input:  [1690, 410, 1691, 0, 17, 76, 1692, 1693, 14, 1694]
Target: [[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]]
Input:  [1691, 0, 17, 76, 1692, 1693, 14, 1694, 1123, 0]
Target: [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  1.]]
Input:  [17, 76, 1692, 1693, 14, 1694, 1123, 0, 6, 1695]
Target: [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


Of course I still need the integer representation, but that will come soon.

# Ideas for the architecture
I can probably use the same network that I have used to classify game tags from game descriptions. I would simply use a slightly different output layer, as described above.

Using Keras, the architecture I will try first will be something like this:

## The initial architecture

In [16]:
model = Sequential()
model.add(Embedding(5000, 128, input_length=10))
model.add(Dropout(0.5))
model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(50))
model.add(Dense(10))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 128)           640000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 64)             41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                23000     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total para

## Training the model

I have to be careful about the test/train/dev split. Right now I am doing it wrong because each sentence is included several times in `X` because many of the sentences will be longer than 10 words. I can solve the problem by splitting into test/train/dev before doing the pre-processing.

In [18]:
X = sequence.pad_sequences(X, maxlen=10)
Y = np.concatenate(Y, axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (7028, 10)
y_train shape: (7028, 10)
X_test shape: (1758, 10)
y_test shape: (1758, 10)


In [19]:
history = model.fit(X_train, y_train,
          batch_size=8,
          epochs=20,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 7028 samples, validate on 1758 samples
Epoch 1/20
7028/7028 [==============================] - 13s - loss: 0.2468 - acc: 0.9269 - val_loss: 0.1195 - val_acc: 0.9695
Epoch 2/20
7028/7028 [==============================] - 12s - loss: 0.1009 - acc: 0.9703 - val_loss: 0.0878 - val_acc: 0.9732
Epoch 3/20
7028/7028 [==============================] - 12s - loss: 0.0817 - acc: 0.9744 - val_loss: 0.0829 - val_acc: 0.9751
Epoch 4/20
7028/7028 [==============================] - 13s - loss: 0.0700 - acc: 0.9771 - val_loss: 0.0844 - val_acc: 0.9728
Epoch 5/20
7028/7028 [==============================] - 14s - loss: 0.0600 - acc: 0.9800 - val_loss: 0.0842 - val_acc: 0.9735
Epoch 6/20
7028/7028 [==============================] - 13s - loss: 0.0504 - acc: 0.9831 - val_loss: 0.0888 - val_acc: 0.9724
Epoch 7/20
7028/7028 [==============================] - 13s - loss: 0.0422 - acc: 0.9856 - val_loss: 0.0929 - val_acc: 0.9722
Epoch 8/20
7028/7028 [==============================] - 12s - loss: 0.

In [20]:
print('Train Loss:', np.min(history.history['loss']))
print('Test Loss:', np.min(history.history['val_loss']))

Train Loss: 0.00745231393997
Test Loss: 0.0828837636981


# pre-processing pipeline

In [22]:
def pre_process_pipeline(string):
    X, Y = pre_process([string])
    X = sequence.pad_sequences(X, maxlen=10)
    return X

In [34]:
example = 'Dette er en sætning som der burde indholde et komma.'
processed = pre_process_pipeline(example)
y_hat = model.predict_proba(processed)

for word, pred in zip(example.split(), y_hat[0]):
    print(word, pred)

#for word, pred in zip(example.split(), y_hat):
#    print(word, pred)

1/1 [==============================] - 0s
Dette 0.00461095
er 0.0194486
en 0.00590108
sætning 0.998058
som 0.00124411
der 0.016666
burde 0.0118635
indholde 0.936682
et 0.00160031
komma. 0.0197901


# Evaluation metrics
Accuracy, precision and recall should be fine for this problem. However, since a sentence can have more than one comma, I think it will be useful to calculate the metrics both for the entire sentence as well as for each comma in the entire text corpus. 

1. **Total recall:** the number of sentences where every comma was placed correctly by the model divided by the total number of sentences with commas in them.
1. **Total precision** The number of sentences with every comma correctly placed by the model divided by the number of sentences predicted to have any number of commas in them.
1. **Recall** The number of correctly placed commas divided by the total number of commas.
1. **Precision** The number of correctly placed commas divided by the number of placed commas. 

# Deployment
The end goal is to implement it in an editor such as VS Code or Atom. But I will probably make a Falsk App first.